In [ ]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime

path = './test/'

## Review 데이터 가공

user_id 및 business_id 추출, 정렬

In [43]:
# 파일 경로 설정
json_file_path = path + 'review.json'

# 유저 ID와 비즈니스 ID를 매핑하기 위한 딕셔너리
user_id_mapping = {}
business_id_mapping = {}
next_user_index = 0
next_business_index = 0

# 최종 데이터를 리스트에 저장
sort_data = []

# JSON 파일을 읽고 처리
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    for line in json_file:
        data = json.loads(line)
        user_id = data['user_id']
        business_id = data['business_id']

        # 유저 ID 매핑 업데이트
        if user_id not in user_id_mapping:
            user_id_mapping[user_id] = next_user_index
            next_user_index += 1
        # 비즈니스 ID 매핑 업데이트
        if business_id not in business_id_mapping:
            business_id_mapping[business_id] = next_business_index
            next_business_index += 1

        # 매핑된 ID로 데이터 저장
        sort_data.append([user_id_mapping[user_id], business_id_mapping[business_id]])

# 데이터를 정렬하고 새 파일에 저장
sort_data.sort(key=lambda x: (x[0], x[1]))  # 먼저 user_id, 다음 business_id 기준으로 정렬

print("모든 데이터가 user_id와 business_id를 기준으로 정렬되어 저장되었습니다.")


모든 데이터가 user_id와 business_id를 기준으로 정렬되어 저장되었습니다.


리뷰가 10개 이상인 유저만 필터링

In [46]:
# sort_data DataFrame으로 변환
sort_data = pd.DataFrame(sort_data, columns=['user_id', 'business_id'])

# 각 ID의 출현 빈도를 계산
frequency = sort_data['user_id'].value_counts()

# 출현 빈도가 10 이상인 ID만 필터링
valid_ids = frequency[frequency >= 10].index

# 필터링된 ID에 해당하는 데이터만 선택
filtered_data = sort_data[sort_data['user_id'].isin(valid_ids)]

# 혹은 출력하여 확인
print(filtered_data)

         user_id  business_id
0              0            0
1              0          473
2              0          927
3              0        15261
4              0        15753
...          ...          ...
6648468  1676360       114399
6648469  1676360       114399
6648470  1676360       114399
6648471  1676360       114399
6648472  1676360       114399

[3303811 rows x 2 columns]


user_id, business_id 인덱스 매핑

In [47]:
# 유저 ID에 대해 고유한 ID를 정렬된 리스트로 추출하고 0부터 시작하는 새로운 인덱스를 매핑
unique_user_ids = sorted(filtered_data['user_id'].unique())
user_id_mapping = {id: index for index, id in enumerate(unique_user_ids)}
filtered_data['user_id'] = filtered_data['user_id'].map(user_id_mapping)

# 비즈니스 ID에 대해 고유한 ID를 정렬된 리스트로 추출하고 0부터 시작하는 새로운 인덱스를 매핑
unique_business_ids = sorted(filtered_data['business_id'].unique())
business_id_mapping = {id: index for index, id in enumerate(unique_business_ids)}
filtered_data['business_id'] = filtered_data['business_id'].map(business_id_mapping)

# 중복 제거
indexing_data = filtered_data.drop_duplicates(subset=['user_id', 'business_id'])

# 최종 데이터 프레임 출력하여 확인
print(indexing_data)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17096\3316432536.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['user_id'] = filtered_data['user_id'].map(user_id_mapping)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17096\3316432536.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['business_id'] = filtered_data['business_id'].map(business_id_mapping)


         user_id  business_id
0              0            0
1              0          471
2              0          923
3              0        14710
4              0        15202
...          ...          ...
6474055   117367       132905
6474056   117367       133473
6507832   117368        98751
6507833   117368       134923
6648457   117369       110433

[3126989 rows x 2 columns]


In [50]:
# drop_duplicates 함수를 사용하여, 각 사용자 ID와 아이템 ID 조합의 첫 번째 등장만 유지합니다.
first_data = indexing_data.drop_duplicates(subset=['user_id', 'business_id'])

# 각 사용자별로 첫 번째 데이터를 테스트 데이터로, 나머지를 트레이닝 데이터로 분리
train_data = []
test_data = []

# 사용자별로 그룹화하여 처리
for user_id, group in indexing_data.groupby('user_id'):
    test_data.append(group.iloc[0].tolist())  # 첫 번째 행을 테스트 데이터에 추가, 리스트로 변환
    train_data.extend(group.iloc[1:].values.tolist())  # 나머지 행을 트레이닝 데이터에 추가, 리스트로 변환

# 리스트를 DataFrame으로 변환
train_df = pd.DataFrame(train_data, columns=['user_id', 'business_id'])
test_df = pd.DataFrame(test_data, columns=['user_id', 'business_id'])

# CSV 파일로 저장
train_df.to_csv(path + 'train.csv', index=False, header=False, sep = '\t')
test_df.to_csv(path + 'test.csv', index=False, header=False, sep = '\t')

print("'train.csv'와 'test.csv' 파일이 생성되었습니다.")

'train.csv'와 'test.csv' 파일이 생성되었습니다.


In [51]:
# 유저와 아이템 집합 구하기
users = indexing_data['user_id'].unique()
businesses = indexing_data['business_id'].unique()

# 결과를 저장할 딕셔너리
neg_dict = {}

# 각 유저마다 가지고 있지 않은 아이템 찾기
for user in users:
    # 현재 유저가 가진 아이템
    user_businesses = indexing_data[indexing_data['user_id'] == user]['business_id'].unique()
    # 가지고 있지 않은 아이템
    not_having_businesses = np.setdiff1d(businesses, user_businesses)
    # 가지고 있지 않은 아이템 중에서 랜덤하게 100개 선택
    if len(not_having_businesses) > 100:
        sampled_businesses = np.random.choice(not_having_businesses, 100, replace=False)
    else:
        sampled_businesses = not_having_businesses
    # 각 항목을 문자열로 변환 후 결과 딕셔너리에 저장
    neg_dict[user] = ','.join([str(item) for item in sampled_businesses])

# 결과를 CSV 파일로 저장
with open('test_neg.csv', 'w') as f:
    for user, businesses in neg_dict.items():
        f.write(f"{businesses}\n")
        
        
print("text_neg 파일이 생성되었습니다")

text_neg 파일이 생성되었습니다


In [52]:
# 필요한 파일을 읽고 쓰기 위해 open 함수를 사용
with open(path + 'test_neg.csv', 'r') as neg_file, open(path + 'test.csv', 'r') as test_file, open(path + 'test_neg_100.csv', 'w') as result_file:
    # 파일에서 각 줄을 읽음
    neg_lines = neg_file.readlines()
    test_lines = test_file.readlines()
    
    # neg_lines와 test_lines의 길이가 같다고 가정하고, 각 줄을 합침
    for neg_line, test_line in zip(neg_lines, test_lines):
        # test_line에서 데이터를 추출하여 형식을 맞춤
        test_index, test_value = test_line.strip().split(',')
        test_tuple = f'({test_index},{test_value})'
        
        # neg_line에서 쉼표를 탭으로 변환
        neg_data = '\t'.join(neg_line.strip().split(','))
        
        # 결과를 파일에 쓰기
        result_line = test_tuple + '\t' + neg_data.strip() + '\n'
        result_file.write(result_line)

# 결과 파일인 'result.txt'가 생성되고 원하는 형태로 데이터가 저장됨


FileNotFoundError: [Errno 2] No such file or directory: './test/test_neg.csv'

In [ ]:
# # 데이터 파일 로드
# df = pd.read_csv('pos.txt', header=None, names=['user_id', 'business_id'])

# # 각 사용자가 가진 아이템의 집합을 만듭니다.
# user_items = df.groupby('user_id')['business_id'].apply(set)

# # 전체 아이템 집합
# all_items = set(df['business_id'])

# # 각 사용자별로 가지고 있지 않은 아이템을 찾아 랜덤하게 100개 선택
# results = {}
# for user, items in user_items.items():
#     not_owned_items = list(all_items - items)
#     if len(not_owned_items) > 100:
#         selected_items = np.random.choice(not_owned_items, 100, replace=False)
#     else:
#         selected_items = not_owned_items
#     results[user] = selected_items

# # 결과를 파일로 저장, 사용자 ID 없이 아이템만 저장
# with open('neg_100.txt', 'w') as file:
#     for items in results.values():
#         items_str = ','.join(map(str, items))
#         file.write(f"{items_str}\n")
